This notebook is aiming at checking detailed distributions of training records. It also checks the number of multi-labeled records compared to signal-labeled records.

Pre-requisite:

1) Training records prepared in Comprehend input required format

Output:

1) Detailed distributions of training records

2) The number of multi-labeled records

In [1]:
# Import required libraires
import numpy as np
import pandas as pd

In [2]:
# Load batch3 training file
batch3_file = pd.read_csv("batch3_combined_multi_label.csv")
batch3_file

content  \
0       !! UNABLE TO FIND 4 tablets every morning. Med...   
1       !! predniSONE 20 MG tabletTake 40mg of prednis...   
2       !06$69608UPS Restrictions00000200 NOSLS LISIA ...   
3       !1Authorization is hereby given to dispense th...   
4       !I, AVILA, ANIBAL, and/or designee, have recei...   
...                                                   ...   
832117  © 2000-2021 The StayWell Company, LLC. All rig...   
832118  © 2016, Elsevier/Gold Standard. (2015-11-18 15...   
832119  àPage: 173PIERCE, HENRY CHARLESFac: St. Joseph...   
832120  àPage: 323PIERCE, HENRY CHARLESFac: St. Joseph...   
832121  ⑇~1356072~51260774~I&O_RPT~##ENHMC VRF ReportL...   

                                                    class  
0       admission_reason|labs|admission_order|discharg...  
1                                       discharge_summary  
2                                                  others  
3                                                  others  
4                                                  others  
...                                                   ...  
832117                                             others  
832118                                             others  
832119                                             others  
832120                                             others  
832121                                             others  

[832122 rows x 2 columns]

In [3]:
# Load batch2 training file
batch2_file = pd.read_csv('training_data_multi_label_phase3_data.csv', names=['class', 'content'])
batch2_file

class                                            content
0                  others  ! 5SOUTH NASSAUINMUNITIES HOSPITALMedical Reco...
1       discharge_summary  ! u 191218905785004Weiss Memorial HospitalPati...
2                  others  !(use first)SOUTH NASSAU COMMUNITIES HOSPITALO...
3               radiology  !.ast Name: TURNERFirst Name: AURTHURPatient I...
4                  others  !06$69608UPS Restrictions00000200 NOSLS LISIA ...
...                   ...                                                ...
677686          radiology  Ürders and Results (continued)SHOULDER 3 VIEWS...
677687             others  ç !Swedish Covenar.t Hospital LIVEPatient Orde...
677688             others  é 85Jun 02 2018 17:14:46 Via Fax9797765333AT&T...
677689             others  édication AlhistrationbrdAdministraçico Dhon12...
677690               labs  ₹10408509197RR0R11CUMULATIVE REPORTPage:220037...

[677691 rows x 2 columns]

In [4]:
# Merge batch2 and batch3 together
merged_df = pd.concat([batch2_file, batch3_file], axis=0, ignore_index=True)
merged_df

class                                            content
0                   others  ! 5SOUTH NASSAUINMUNITIES HOSPITALMedical Reco...
1        discharge_summary  ! u 191218905785004Weiss Memorial HospitalPati...
2                   others  !(use first)SOUTH NASSAU COMMUNITIES HOSPITALO...
3                radiology  !.ast Name: TURNERFirst Name: AURTHURPatient I...
4                   others  !06$69608UPS Restrictions00000200 NOSLS LISIA ...
...                    ...                                                ...
1509808             others  © 2000-2021 The StayWell Company, LLC. All rig...
1509809             others  © 2016, Elsevier/Gold Standard. (2015-11-18 15...
1509810             others  àPage: 173PIERCE, HENRY CHARLESFac: St. Joseph...
1509811             others  àPage: 323PIERCE, HENRY CHARLESFac: St. Joseph...
1509812             others  ⑇~1356072~51260774~I&O_RPT~##ENHMC VRF ReportL...

[1509813 rows x 2 columns]

In [5]:
# Create count variable to check multi-labeled records, create dataframe to save separated multi-labeled records
multi_label_count = 0
new_df = pd.DataFrame()

Main part to separate multi-labeled records to signle label, and counting the number of multi-labeled records

In [14]:
# new_df will have all training records with single-label 
for index in range(len(merged_df)):
    class_label = merged_df['class'].iloc[index]
    segment = class_label.split('|')
    if len(segment) >=2:
        multi_label_count = multi_label_count + 1
        for each_class in segment:
            segment_df = pd.DataFrame([{'class' : each_class}])
            new_df = pd.concat([new_df, segment_df], axis=0, ignore_index=True)
    else:
        segment_df = pd.DataFrame([{'class' : class_label}])
        new_df = pd.concat([new_df, segment_df], axis=0, ignore_index=True)

In [17]:
# Check the detailed distributions of training records
new_df['class'].value_counts()

others                    893260
flowsheets                187821
progress_notes            110888
mar                        83369
labs                       76627
consult                    31722
emergency_department       31686
radiology                  30456
history_physical           27823
discharge_summary          25935
admission_order            13219
admission_reason            9794
admit_type                  8862
procedures                  6363
discharge_disposition       1351
physician_query             1345
case_management             1105
nutritional_assessment       763
nutrition_assessment          26
Name: class, dtype: int64

In [18]:
# Check the number of multi-labeled records
multi_label_count

30286

In [20]:
# Check the number of all useful training records (records not in 'others' class)
len(new_df['class']) - len(new_df[new_df['class']=='others'])

649155